In [1]:
import json
import numpy as np
from skimage.transform import resize
from time import sleep
import matplotlib.pyplot as plt

In [2]:
#Read information json:
PATH_JSON_INFO = '/Datasets/PICAI_olmos/info-12x32x32.json'
PATH_VOLS = '/Datasets/PICAI_32x32x12/volumes/'
#read json

with open(PATH_JSON_INFO, 'r') as f:
    info = json.load(f)

In [3]:
#make a function that given a volume returns the center slice in x,y and z
def get_center_slices(vol):
    x,y,z = vol.shape
    return vol[(x-1)//2,:,:], vol[:,(y-1)//2,:], vol[:,:,(z-1)//2]

def resize2_32(slice):
    return resize(slice, (32,32), anti_aliasing=True)

def resize2_224(slice):
    return resize(slice, (224,224), anti_aliasing=True)

def get_spd(acts,type="gramm"):
    h,w,d = acts.shape
    vect_acts = acts.reshape(h*w,d)
    if type == "gramm":
        spd = vect_acts.T@vect_acts
    elif type == "corr":
        spd = np.corrcoef(vect_acts.T)
    elif type == "cov":
        spd = np.cov(vect_acts.T)
    return spd

In [4]:
#Using tensorflow and VGG9 describe the slices
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

#Load the model
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(32,32,3))

#Get the first pooling layer
model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block1_conv2').output)
model.summary()


2024-03-18 20:41:38.589081: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 20:41:56.717903: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-03-18 20:41:56.717967: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: 99f8986afac3
2024-03-18 20:41:56.717975: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: 99f8986afac3
2024-03-18 20:41:56.718074: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 545.23.6
2024-03-18 20:41:56.718095: I external/local_xla/xla

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,720 (151.25 KB)

 Trainable params: 38,720 (151.25 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
PATH_SAVE = '/Datasets/PICAI_32x32x12/covs_vgg19/'
i = 0
long = len(info)
for key in info:
    vol1 = np.load(PATH_VOLS + key+ "_0000" + '.npy')
    vol2 = np.load(PATH_VOLS + key+ "_0001" + '.npy')
    vol3 = np.load(PATH_VOLS + key+ "_0002" + '.npy')

    #Standarize the volumes
    vol1 = (vol1 - vol1.mean()) / vol1.std()
    vol2 = (vol2 - vol2.mean()) / vol2.std()
    vol3 = (vol3 - vol3.mean()) / vol3.std()

    vol_list = [vol1, vol2, vol3]
    s1_list = []
    s2_list = []
    s3_list = []
    for vol in vol_list:
        s1,s2,s3 = get_center_slices(vol)
        s1 = resize2_32(s1)
        s2 = resize2_32(s2)

        #Repeat slices 3 times along 2 axis
        s1 = np.repeat(s1[:,:,np.newaxis], 3, axis=2)
        s2 = np.repeat(s2[:,:,np.newaxis], 3, axis=2)
        s3 = np.repeat(s3[:,:,np.newaxis], 3, axis=2)
        s1_list.append(s1)
        s2_list.append(s2)
        s3_list.append(s3)

    #Stack slices list over first axis
    s1_l = np.stack(s1_list, axis=0)#shape(3 modalities,32,32,3 (RGB))
    s2_l = np.stack(s2_list, axis=0)
    s3_l = np.stack(s3_list, axis=0)

    s1_acts = model.predict(s1_l,verbose=0)
    s2_acts = model.predict(s2_l,verbose=0)
    s3_acts = model.predict(s3_l,verbose=0)

    s1_reordered = s1_acts.transpose(1,2,3,0).reshape(32,32,-1) #shape(32,32,3*64) = (32,32,192) -> COrrelacionando para un slice las 3 modalidades
    s2_reordered = s2_acts.transpose(1,2,3,0).reshape(32,32,-1)
    s3_reordered = s3_acts.transpose(1,2,3,0).reshape(32,32,-1)

    mat = "cov"
    s1_spd = get_spd(s1_reordered,mat)
    s2_spd = get_spd(s2_reordered,mat)
    s3_spd = get_spd(s3_reordered,mat)

    #Save spds as .npy
    np.save(PATH_SAVE + key + "_s1" + ".npy", s1_spd)
    np.save(PATH_SAVE + key + "_s2" + ".npy", s2_spd)
    np.save(PATH_SAVE + key + "_s3" + ".npy", s3_spd)
    
    i = i+1
    #Print loading bar
    print(f"Process {i}/{long}  - ({round((i/long)*100,3)}% done)", end="\r")
    sleep(0.05)

# GEt slices only

In [4]:
PATH_VOLS

'/Datasets/PICAI_32x32x12/volumes/'

In [9]:
PATH_SAVE = '/Datasets/PICAI_32x32x12/slices/'
i = 0
long = len(info)
s_names_list = []
for key in info:

    name1 = key + "_0000"
    name2 = key + "_0001"
    name3 = key + "_0002"
    
    vol1 = np.load(PATH_VOLS + name1 + '.npy')
    vol2 = np.load(PATH_VOLS + name2 + '.npy')
    vol3 = np.load(PATH_VOLS + name3 + '.npy')

    #Standarize the volumes
    vol1 = (vol1 - vol1.mean()) / vol1.std()
    vol2 = (vol2 - vol2.mean()) / vol2.std()
    vol3 = (vol3 - vol3.mean()) / vol3.std()

    vol_list = [vol1, vol2, vol3]
    names_list = [name1, name2, name3]

    for vol,name in zip(vol_list,names_list):
        s1,s2,s3 = get_center_slices(vol)
        s1 = resize2_32(s1)
        s2 = resize2_32(s2)

        #Save slice with identiffier si
        s1_name = PATH_SAVE + name + "_s0" + ".npy"
        s2_name = PATH_SAVE + name + "_s1" + ".npy"
        s3_name = PATH_SAVE + name + "_s2" + ".npy"

        np.save(s1_name, s1)
        np.save(s2_name, s2)
        np.save(s3_name, s3)

        s_names_list = s_names_list + [s1_name, s2_name, s3_name]

    i = i+1
    #Print loading bar
    print(f"Process {i}/{long}  - ({round((i/long)*100,3)}% done)", end="\r")
    sleep(0.05)

In [10]:
s_names_list

['/Datasets/PICAI_32x32x12/slices/11107_1001130_0000_s0.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0000_s1.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0000_s2.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0001_s0.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0001_s1.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0001_s2.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0002_s0.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0002_s1.npy',
 '/Datasets/PICAI_32x32x12/slices/11107_1001130_0002_s2.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0000_s0.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0000_s1.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0000_s2.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0001_s0.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0001_s1.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0001_s2.npy',
 '/Datasets/PICAI_32x32x12/slices/10078_1000078_0002_s0.npy',
 '/Datas